# 1. Hidden Markov Model
#### a.) Assuming you do not know the code beforehand -- i.e., your task is to decrypt a message -- describe what the HMM would look like, in order to decode such words. How many and which states will the HMM have, and what do the emissions look like?

The HMM should have 26 hidden states and 26 observed states, i.e. 1 for each letter
The observed states would be the emission of the hidden states in the HMM.

#### b.) Make a suggestion for how the emission probabilities should be distributed for a well-trained HMM.

For a well-trained HMM the emission probabilities should be as follows:  

P(Hidden = B|Observed = U) = 1  
P(Hidden = A|Observed = C) = 1   
P(Hidden = N|Observed = P) = 1  

and so on. This is because for a well-trained HMM, as it is a simple replacement of characters, one character will be mapped to one character only, the other probabilities like for instance P(Hidden = A|Observed = U) will be 0 or close to 0 in an ideal HMM.

#### c.) Would it make sense to convert the HMM to a trigram model instead of a bigram model? What are the advantages, what are the disadvantages?

##### Advantage
A trigram model allows you to look at two consecutive hidden states to determine the observed state. This allows you to determine the observed state with more confidence than a bigram model. This is becuase we are determining the probability of observed state 2 given the hidden state 2 and hidden state 1.

##### Disadvantage
As this is a simple substitution cryptography, the observed given the hidden need not depend on the previous hidden state. As this won't be a language model we would do extra unnecessary computation.

##### References
1. Lee, Dar-Shyang. "Substitution deciphering based on HMMs with applications to compressed document processing." IEEE Transactions on Pattern Analysis and Machine Intelligence 24.12 (2002): 1661-1666.
2. Vobbilisetty, Rohit, "Cryptanalysis of Classic Ciphers Using Hidden Markov Models" (2015). Master's Projects. 407.

## 2. Forward/Backward Algorithm

In [1]:
import numpy as np

In [2]:
sentence = " a myth is a female moth"
words = sentence.split()
print("Sentence split into words with indices starting from 0:", words[:6])
observed = np.array([0,1,2,0,3,4])
transition_matrix = np.array(((0.03, 0.42, 0.50, 0.05), (0.01, 0.25, 0.65, 0.09 ) ,(0.07, 0.03, 0.15, 0.75 ), (0.30, 0.25, 0.15, 0.30) ))
emission_matrix = np.array(((0.84, 0.01, 0.02, 0.01, 0.12), (0.05, 0.10, 0.02, 0.70, 0.13), (0.03, 0.45, 0.02, 0.25, 0.25 ), (0.05, 0.10, 0.60, 0.05, 0.20) ))
initial_vector = np.array((0.45, 0.35, 0.15, 0.05))

Sentence split into words with indices starting from 0: ['a', 'myth', 'is', 'a', 'female', 'moth']


In [3]:
def forward(observed, transition_matrix, emission_matrix, initial_vector):
    alpha = np.zeros((observed.shape[0], transition_matrix.shape[0]))
    alpha[0, :] = initial_vector * emission_matrix[:, observed[0]]
    for t in range(1, observed.shape[0]):
        for j in range(transition_matrix.shape[0]): 
          alpha[t, j] = alpha[t - 1].dot(transition_matrix[:, j]) * emission_matrix[j, observed[t]]
    return alpha

In [4]:
alpha = forward(observed, transition_matrix, emission_matrix, initial_vector)
print("Forward probabilites are :")
print(alpha)

Forward probabilites are :
[[3.78000000e-01 1.75000000e-02 4.50000000e-03 2.50000000e-03]
 [1.25800000e-04 1.63895000e-02 9.06412500e-02 2.46000000e-03]
 [1.45011130e-04 1.49688970e-04 4.93625250e-04 4.21201695e-02]
 [1.06482195e-02 5.32158902e-04 1.96856178e-04 6.51349618e-04]
 [5.33952993e-06 3.34182452e-03 1.44931098e-03 4.61676149e-05]
 [1.78656581e-05 1.16053595e-04 5.99794373e-04 2.80372941e-04]]


In [5]:
def backward(observed, transition_matrix, emission_matrix):
    beta = np.zeros((observed.shape[0], transition_matrix.shape[0])) 
    beta[observed.shape[0] - 1] = np.ones((transition_matrix.shape[0])) 
    for t in range(observed.shape[0] - 2, -1, -1):
        for j in range(transition_matrix.shape[0]):
            beta[t, j] = (beta[t + 1] * emission_matrix[:, observed[t + 1]]).dot(transition_matrix[j, :]) 
    return beta

In [6]:
beta = backward(observed, transition_matrix, emission_matrix)
print("Backward probabilites are :")
print(beta)

Backward probabilites are :
[[2.51427560e-03 3.23782233e-03 1.06266783e-03 8.98597264e-04]
 [8.11942904e-04 1.39966112e-03 1.08150547e-02 4.37179056e-03]
 [4.10873062e-03 2.20079171e-03 7.18968200e-03 2.39698065e-02]
 [8.84227600e-02 7.07188200e-02 1.83509400e-02 4.80471000e-02]
 [1.93200000e-01 2.14200000e-01 1.99800000e-01 1.66000000e-01]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]]


In [7]:
print("Following are the forward & backward probability for given time steps and states: ")
print("Alpha_4 (NN) :", round((alpha[3][2]),8))
print("Alpha_3 (VB) :", round((alpha[2][3]),8))
print("Alpha_1 (DT) :", round((alpha[0][0]),8))
print("Beta_4  (NN) :", round((beta[3][2]) ,8))
print("Beta_2  (NN) :", round((beta[1][2]) ,8))

Following are the forward & backward probability for given time steps and states: 
Alpha_4 (NN) : 0.00019686
Alpha_3 (VB) : 0.04212017
Alpha_1 (DT) : 0.378
Beta_4  (NN) : 0.01835094
Beta_2  (NN) : 0.01081505


#### References http://www.adeveloperdiary.com/data-science/machine-learning/forward-and-backward-algorithm-in-hidden-markov-model/

#### Note: Assignment Discussed with Chaitanya Patil and Animesh Sagar